In [1]:
"""Model selection using instrument features and properties

Demonstration of how to configure model selectio for valuation based on instrument features as well as instrument properties.

Attributes
----------
Instruments
Derived instrument properties
Model selection
Instrument features
"""

'Model selection using instrument features and properties\n\nDemonstration of how to configure model selectio for valuation based on instrument features as well as instrument properties.\n\nAttributes\n----------\nInstruments\nDerived instrument properties\nModel selection\nInstrument features\n'

# Model selection using instrument features and properties

In this notebook, we demonstrate how models can be chosen within configuration recipe in LUSID. We show how one can select model for an instrument based on instrument features as well as (derived) instrument properties. Here the instrument features are labels attached onto an instrument (by the system) outlining its features for example an FX option might have feature of category ExerciseType with value American or feature of category DeliveryType with value Physical. Instrument properties on the other hand are labels assigned by the user themselves on per instrument basis. This allows a myriad of ways to control how model selection occurs in the valuation call. Instrument features can be seen from the UI but they can also be requested via API endpoint, this and more about the features is outlined in section 7.

In [2]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
import json
import pytz
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
import lusid.api as la
from lusid.utilities import ApiClientFactory

# Import key functions from Lusid-Python-Tools and other packages
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory : ApiClientFactory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path)

print ('LUSID Environment Initialised')
print ('LUSID API Version :', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.6.11339.0


In [3]:
# Set required APIs
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
properties_api = api_factory.build(lusid.PropertyDefinitionsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)

In [4]:
# Define global variables
scope = "ibor-opt-vmr"
market_data_scope = "ibor-opt-vmr"
market_supplier = "Lusid"
portfolio_code = "pf_code-opt-vmr"
recipe_code = "OptValuation-vmr"

# 1. Create Portfolio

We begin by creating a portfolio that will contain the FX option instruments that we will be looking to price. We will be grouping the OTC deals using a sub-holding key, this will then keep the transactions associated with a particular deal (and not instrument which is the default LUSID behaviour) linked.

In [5]:
# Obtain the LUSID Property Definition API
property_definition_api = api_factory.build(la.PropertyDefinitionsApi)

# Create a property definition for the SHK in the 'Transaction' domain, with a unique scope and code
property_definition = lm.CreatePropertyDefinitionRequest(
    domain = "Transaction",
    scope = scope,
    code = "DealId",
    display_name = "Deal ID",
    data_type_id = lm.ResourceId(
        scope = "system",
        code = "string"
    )
)

# Upsert property definition to LUSID
try:
    upsert_property_definition_response = property_definition_api.create_property_definition(
        create_property_definition_request = property_definition
    )
    print(f"Property definition created with the following key: {upsert_property_definition_response.key}")
except lusid.ApiException as e:
    if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            print(f"Property definition with the following key already exists: {property_definition.domain}/{property_definition.scope}/{property_definition.code}")


Property definition with the following key already exists: Transaction/ibor-opt-vmr/DealId


In [6]:

try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="EUR",
            created="2010-01-01",
            instrument_scopes=[scope]
        ),
    )

except lusid.ApiException as e:
    if json.loads(e.body)['name'] == "PortfolioWithIdAlreadyExists":
        print("Portfolio with this id already exists.")
    else:
        raise e

Portfolio with this id already exists.


# 2. Create Instrument

We create a helper function to create and upsert the fxOption instrument into LUSID, this will reference the SDK models to populate the required parameters for creating an FX Option. We then populate the instrument variables, and make a call to persist the instrument in LUSID.

We are creating 3 fx options with transactions to demonstrate valuation with advanced model selection. The three instruments we will be inserting are as follows:

1. Option_Special_EUR:  A 1M vanilla european option with instrument property "Instrument/{scope}/special" set to "yes",
2. Option_Not_Special_EUR:  A 4M vanilla european option with instrument property "Instrument/{scope}/special" set to "no",
3. Option_AMER:  A 2M vanilla american option with no instrument property set.

Instrument properties and instrument features can be used to select models used for instrument pricing. Demonstrating how this is done is the main intention behind this notebook this is why "Instrument/{scope}/special" property is being created here and attached to some instruments.

In [7]:
# Define function that creates an FX option

def create_fx_option(strike, dom_ccy, fgn_ccy, start_date, maturity_date, settlement_date, is_call, is_fx_delivery = True, is_payoff_digital = False, exercise_type : str = "European"):
    
    return lm.FxOption(
        strike = strike,
        dom_ccy = dom_ccy,
        fgn_ccy = fgn_ccy,
        start_date = start_date,
        option_maturity_date = maturity_date,
        option_settlement_date = settlement_date,
        is_call_not_put = is_call,
        is_delivery_not_cash = is_fx_delivery,
        is_payoff_digital = is_payoff_digital,
        exercise_type = exercise_type,
        instrument_type = "FxOption",
        dom_amount = 1
    )

# Define a function to upsert instrument
def upsert_instrument_to_lusid(instr_scope, instrument_definition, name, identifier, identifier_type, props = None):
        return instruments_api.upsert_instruments(
            request_body={
                identifier: lm.InstrumentDefinition(
                    name=name,
                    identifiers={
                        identifier_type: lm.InstrumentIdValue(value=identifier)
                    },
                    definition=instrument_definition,
                    properties=props
                )
            },
            scope = instr_scope
        )

In [8]:
# Set common  trade variables
trade_date = datetime(2021, 1, 20, tzinfo=pytz.utc)
start_date = trade_date
settle_days = 2
units = 1000000
dom_ccy = "EUR"
fgn_ccy = "USD"
strike = 1.2050
premium_currency = "EUR"
identifier_type = "ClientInternal"

In [9]:
# Create instrument property

properties = lm.ModelProperty(key=f"Instrument/{scope}/special")
try:
    properties_api.create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code="special",
            display_name="special",
            data_type_id=lm.ResourceId(code="string", scope="system"),
        )
    )
except lusid.ApiException as e:
    if json.loads(e.body)["name"] == "PropertyAlreadyExists":
        print("Such property is already upserted.")
    else:
        raise e

Such property is already upserted.


In [10]:
# 1. Option_Special_EUR:  A 1M vanilla european option with instrument property "Instrument/{scope}/special" set to "yes"
maturity_date = trade_date + timedelta(days = 30)
settlement_date = maturity_date + timedelta(days = 2)

option_special_eur_name = "EUR/USD European Special FX Option " + maturity_date.strftime("%m/%d/%Y")  + " " + str(strike)
option_special_eur_identifier = "EURUSDEuropeanSpecialOptionDemo"

# Create the Instrument
option_special_eur_definition = create_fx_option(
    strike = strike,
    dom_ccy = dom_ccy,
    fgn_ccy = fgn_ccy,
    start_date = start_date,
    maturity_date = maturity_date,
    settlement_date = settlement_date,
    is_call = True,
    is_fx_delivery = True,
    is_payoff_digital = False,
    exercise_type = "European"
)

# Upsert the instrument
upsert_response = upsert_instrument_to_lusid(
    scope,
    option_special_eur_definition,
    option_special_eur_name,
    option_special_eur_identifier,
    identifier_type,
    [lm.ModelProperty(f"Instrument/{scope}/special", value=lm.PropertyValue(label_value="yes"))]
)

luid_opt1 = upsert_response.values[option_special_eur_identifier].lusid_instrument_id
print(f"{luid_opt1} : {option_special_eur_identifier}")

LUID_00078YQK : EURUSDEuropeanSpecialOptionDemo


In [11]:
# 2. Option_Not_Special_EUR:  A 4M vanilla european option with instrument property "Instrument/{scope}/special" set to "no"
maturity_date = trade_date + timedelta(days = 120)
settlement_date = maturity_date + timedelta(days = 2)

option_not_special_eur_name = "EUR/USD European Not Special FX Option " + maturity_date.strftime("%m/%d/%Y")  + " " + str(strike)
option_not_special_eur_identifier = "EURUSDEuropeanNotSpecialOptionDemo"

# Create the Instrument
option_not_special_eur_definition = create_fx_option(
    strike = strike,
    dom_ccy = dom_ccy,
    fgn_ccy = fgn_ccy,
    start_date = start_date,
    maturity_date = maturity_date,
    settlement_date = settlement_date,
    is_call = True,
    is_fx_delivery = True,
    is_payoff_digital = False,
    exercise_type = "European"
)

# Upsert the instrument
upsert_response = upsert_instrument_to_lusid(
    scope,
    option_not_special_eur_definition,
    option_not_special_eur_name,
    option_not_special_eur_identifier,
    identifier_type,
    [lm.ModelProperty(f"Instrument/{scope}/special", value=lm.PropertyValue(label_value="no"))]
)

luid_opt2 = upsert_response.values[option_not_special_eur_identifier].lusid_instrument_id
print(f"{luid_opt2} : {option_not_special_eur_identifier}")

LUID_00078YQL : EURUSDEuropeanNotSpecialOptionDemo


In [12]:
# 3. Option_AMER:  A 2M vanilla american option with no instrument property set.
maturity_date = trade_date + timedelta(days = 60)
settlement_date = maturity_date + timedelta(days = 2)

option_amer_name = "EUR/USD American FX Option " + maturity_date.strftime("%m/%d/%Y")  + " " + str(strike)
option_amer_identifier = "EURUSDAmericanOptionDemo"

# Create the Instrument
option_amer_definition = create_fx_option(
    strike = strike,
    dom_ccy = dom_ccy,
    fgn_ccy = fgn_ccy,
    start_date = start_date,
    maturity_date = maturity_date,
    settlement_date = settlement_date,
    is_call = True,
    is_fx_delivery = True,
    is_payoff_digital = False,
    exercise_type = "American"
)

# Upsert the instrument
upsert_response = upsert_instrument_to_lusid(
    scope,
    option_amer_definition,
    option_amer_name,
    option_amer_identifier,
    identifier_type
)

luid_opt3 = upsert_response.values[option_amer_identifier].lusid_instrument_id
print(f"{luid_opt3} : {option_amer_identifier}")

LUID_00078YQM : EURUSDAmericanOptionDemo


# 3. Upsert Transactions

Having created the portfolio and the Instruments, we add a Buy transaction against the option to create a position with the costs incurred representing the premium for the option.

In [13]:
premium = 0.02

txs = {
    "TXN001": option_special_eur_identifier,
    "TXN002": option_not_special_eur_identifier,
    "TXN003": option_amer_identifier
}
for tx in txs:
    opt_txn = lm.TransactionRequest(
        transaction_id= tx,
        type="Buy",
        instrument_identifiers={"Instrument/default/ClientInternal": txs[tx]},
        transaction_date=trade_date.isoformat(),
        settlement_date=(trade_date + timedelta(days = settle_days)).isoformat(),
        units=units,
        transaction_price=lm.TransactionPrice(price=premium,type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=premium*units,currency=premium_currency),
        exchange_rate=1,
        transaction_currency=premium_currency
    )

    response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                        code=portfolio_code,
                                                        transaction_request=[opt_txn])

    print(f"Transaction successfully updated at time: {response.version.as_at_date}")

Transaction successfully updated at time: 2023-05-15 15:42:59.853748+00:00
Transaction successfully updated at time: 2023-05-15 15:42:59.853748+00:00
Transaction successfully updated at time: 2023-05-15 15:42:59.853748+00:00


# 4. Upsert Market Data

Next we will be loading the required marked data into the quotes store. In this particular example to value an option, we will be using the standard Black-Scholes pricing model. To understand what economic dependencies we may call instrument capabilities endpoint for guidance, for example for the first european fx option if we want to use BlackScholes model we need the following economic dependencies:

In [14]:
capabilities = instruments_api.get_existing_instrument_capabilities(identifier=luid_opt1, model="BlackScholes", instrument_scope=scope) # Here luid_opt1 corresponds to "EUR/USD European Special FX Option"
df = pd.DataFrame(capabilities.economic_dependencies)

display(df)

,0
0,{'dependency_type': 'Fx'}
1,{'dependency_type': 'Fx'}
2,{'dependency_type': 'Cash'}
3,{'dependency_type': 'Cash'}
4,{'dependency_type': 'Discounting'}


In particular, we require the following data:
1. EUR/USD Spot Rates - this is the value of the underlying of the option
2. EUR and USD discount curves - these curves are required for 2 things: to calculate the implied fx forward rate; and as the option is expiring and maturing in the future, to discount future cash flows
3. EURUSD volatility surface - the value of an option is closely related to the expected volatility of the underlying over the period of the option

## 4.1 FX Spot Rates

We begin by adding the FX spot rates.

In [15]:
# Read fx spot rates and make datetimes timezone aware
quotes_df = pd.read_csv("data/eurusd_spot.csv")
quotes_df["Date"] = pd.to_datetime(quotes_df["Date"], dayfirst=True)
quotes_df["Date"] = quotes_df["Date"].apply(lambda x: x.replace(tzinfo=pytz.utc))
quotes_df.head()

,Date,Rate,Pair
0,2021-01-01 00:00:00+00:00,1.2215,EUR/USD
1,2021-01-04 00:00:00+00:00,1.2248,EUR/USD
2,2021-01-05 00:00:00+00:00,1.2298,EUR/USD
3,2021-01-06 00:00:00+00:00,1.2327,EUR/USD
4,2021-01-07 00:00:00+00:00,1.2272,EUR/USD


In [16]:
# Create quotes request
instrument_quotes = {
            index: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider=market_supplier,
                    instrument_id=row["Pair"],
                    instrument_id_type="CurrencyPair",
                    quote_type="Rate",
                    field="mid",
                ),
                effective_at=row["Date"].isoformat(),
            ),
            metric_value=lm.MetricValue(value=row["Rate"], unit=row["Pair"]),
        )
    for index, row in quotes_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=market_data_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 223 quotes loaded.


## 4.2 EUR & USD Discount curves

Next we will read in the EUR and USD Discount factors and create separate OIS curves.

In [17]:
# Read FX market data - showing first 10 terms
df_market_data = pd.read_csv("data/EURUSD-MarketData.csv")
df_market_data["Days"]= df_market_data["Days"].astype(int)
df_market_data = df_market_data.reset_index()
df_market_data.head(10)

,index,Tenor,Days,EUR-Rates,USD-Rates,EURdf,USDdf,Vol25D,ATM,Vol75D,Strike25D,StrikeATM,Strike75D
0,0,1M,30,0.2200,1.2200,0.9998,0.9990,0.1200,0.1000,0.1100,1.1850,1.2000,1.2150
1,1,2M,60,0.3700,1.3700,0.9994,0.9978,0.1205,0.1010,0.1105,1.1850,1.2000,1.2150
2,2,3M,90,0.5300,1.5300,0.9987,0.9962,0.1210,0.1020,0.1110,1.1850,1.2000,1.2150
3,3,6M,180,0.9500,1.9500,0.9953,0.9904,0.1215,0.1030,0.1115,1.1850,1.2000,1.2150
4,4,1Y,365,1.3000,2.3000,0.9871,0.9773,0.1220,0.1040,0.1120,1.1850,1.2000,1.2150
5,5,18M,547,1.5000,2.5000,0.9778,0.9632,0.1225,0.1050,0.1125,1.1850,1.2000,1.2150
6,6,2Y,730,1.5000,2.5000,0.9705,0.9513,0.1230,0.1060,0.1130,1.1850,1.2000,1.2150
7,7,3Y,1095,1.4300,2.4300,0.9580,0.9297,0.1235,0.1070,0.1135,1.1850,1.2000,1.2150
8,8,4Y,1460,1.3700,2.3700,0.9467,0.9096,0.1240,0.1080,0.1140,1.1850,1.2000,1.2150
9,9,5Y,1825,1.3100,2.3100,0.9366,0.8909,0.1245,0.1090,0.1145,1.1850,1.2000,1.2150


We will now upsert discount factors, in particular we will use the `DiscountFactorCurveData` market data type. For information about what categories of complex market data types are available please refer to https://support.lusid.com/knowledgebase/article/KA-02004/en-us.

In [18]:
# Set up the OIS curves loader function
def upsert_discount_factors(scope, effective_at, market_asset, dates, dfs):

    complex_market_data = lm.DiscountFactorCurveData(
        base_date=effective_at,
        dates=dates,
        discount_factors = dfs,
        market_data_type="DiscountFactorCurveData"
    )

    # create a unique identifier for our OIS yield curves
    complex_id = lm.ComplexMarketDataId(
        provider=market_supplier,
        price_source = market_supplier,
        effective_at=effective_at,
        market_asset=market_asset,
    )

    upsert_request = lm.UpsertComplexMarketDataRequest(
        market_data_id=complex_id, market_data=complex_market_data
    )


    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope, request_body={market_asset: upsert_request}
    )

    if response.failed:
        print(f"Failed to upload yield curve {response.failed}")

    print(f"{market_asset} yield curve uploaded into scope={scope}")

    return complex_id

In [19]:
# Set up the OIS curves
base_date = trade_date
dates = []
dfs = []

# loading the data for EUR OIS discount curve
for index, row in df_market_data.iterrows():
    dates.append(base_date + timedelta(days=row["Days"]))
    dfs.append(row["EURdf"])

curve_id_EUR = upsert_discount_factors(market_data_scope, base_date, "EUR/EUROIS", dates, dfs)


dates = []
dfs = []

# loading the data for USD OIS discount curve
for index, row in df_market_data.iterrows():
    dates.append(base_date + timedelta(days=row["Days"]))
    dfs.append(row["USDdf"])
    
curve_id_USD = upsert_discount_factors(market_data_scope, base_date, "USD/USDOIS", dates, dfs)

EUR/EUROIS yield curve uploaded into scope=ibor-opt-vmr
USD/USDOIS yield curve uploaded into scope=ibor-opt-vmr


## 4.3 Create the curve instruments

In this section we create a volatility surface, which is constructed using FX option instruments representing different points in terms of maturity, strike and implied volatility.

In [20]:
def upsert_fxoption_curve_instruments (df_vol_data):

    for index, row in df_vol_data.iterrows():

        option_expiry = base_date + timedelta(days=row["Days"])

        # Create the Instrument
        option_definition = create_fx_option(
            strike = row["Strike"],
            dom_ccy = dom_ccy,
            fgn_ccy = fgn_ccy,
            start_date = base_date,
            maturity_date = option_expiry,
            settlement_date =option_expiry + timedelta(days=2),
            is_call = True,
            is_payoff_digital = False,
            is_fx_delivery = True,
        )

        option_name = "EUR/USD " + " FX Option " + option_expiry.strftime("%m/%d/%Y")
        option_identifier = (f"EURUSDVolCurve{index}:{row['Strike']}")

        curve_map.append({option_name : option_identifier})
        curve_instruments.append(option_definition)

        # Upsert the instrument
        upsert_response = upsert_instrument_to_lusid(scope, option_definition, option_name, option_identifier, identifier_type, None)
        curve_instruments_id.append(upsert_response.values[option_identifier].lusid_instrument_id)

In [21]:
curve_instruments = []
curve_instruments_id = []
curve_map = []

upsert_fxoption_curve_instruments (df_market_data[["Days", "Strike25D"]].rename(columns={"Strike25D" : "Strike"}))
upsert_fxoption_curve_instruments (df_market_data[["Days", "StrikeATM"]].rename(columns={"StrikeATM" : "Strike"}))
upsert_fxoption_curve_instruments (df_market_data[["Days", "Strike75D"]].rename(columns={"Strike75D" : "Strike"}))

print(f"{len(curve_instruments)} instruments uploaded to create the vol curve")

30 instruments uploaded to create the vol curve


## 4.4 Upload Volatility surface

Next we create the volatility surface in LUSID, associating volatility quotes with each of the instruments we have loaded.

In [22]:
# Function to load the surface into Lusid
def upsert_fx_vol(scope, effective_at, market_asset, instruments, vols_low_strike, vols_atm, vols_high_strike):

    market_data_id = lm.ComplexMarketDataId(
        provider=market_supplier,
        price_source="Lusid",
        effective_at=effective_at,
        market_asset=market_asset,
    )
    
    quotes = []
    
    for vol in vols_low_strike:
        quotes.append(lm.MarketQuote(quote_type="LogNormalVol", value=vol))
        
    for vol in vols_atm:
        quotes.append(lm.MarketQuote(quote_type="LogNormalVol", value=vol))   
                
    for vol in vols_high_strike:
        quotes.append(lm.MarketQuote(quote_type="LogNormalVol", value=vol))   

    fx_vol_surface_data = lm.FxVolSurfaceData(
        base_date=effective_at,
        instruments=instruments,
        quotes=quotes,
        market_data_type="FxVolSurfaceData"
    )  
    
    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope,
        request_body={
            market_asset: lm.UpsertComplexMarketDataRequest(
                market_data_id=market_data_id, market_data=fx_vol_surface_data
            )
        },
    )
    
    if len(response.failed) == 0:
        print(f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded.")

    else:
        print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")
        
    return market_data_id

In [23]:
# Actual load of the data into LUSID
EURUSD_vol_curve_id = upsert_fx_vol(market_data_scope, base_date, "EUR/USD/LN", curve_instruments, df_market_data["Vol25D"], df_market_data["ATM"], df_market_data["Vol75D"])

Quote successfully loaded into LUSID. 1 quotes loaded.


# 5. Create Configuration Recipe

In order to run a valuation, LUSID will require a recipe that drives how a valuation will be conducted. The below recipe defines what model and market data to use in this context. The aim of this notebook is to demonstrate model selection, this will present itself below as `model_rules`. Each instrument is checked against each rule starting from top. A rule is satisfied if all the property filters are satisfied, the supplier and the instrument type matches. If a rule is satisfied the model stated is selected, the model selection for that instrument ends there, rules further down are not checked for that instrument.

You can find out more about the valuation [here](https://support.lusid.com/knowledgebase/article/KA-01729/).

In [24]:
# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                    key="FX.CurrencyPair.*",
                    supplier=market_supplier,
                    data_scope=market_data_scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="100D"
                ),
            lm.MarketDataKeyRule(
                key="FXVol.*.*.*",
                supplier=market_supplier,
                data_scope=market_data_scope,
                price_source=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="100D",
            ),
            lm.MarketDataKeyRule(
                key="Rates.*.*",
                supplier=market_supplier,
                data_scope=market_data_scope,
                price_source=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="100D",
            ),
        ],
        options=lm.MarketOptions(
            default_scope = market_data_scope,
            attempt_to_infer_missing_fx=True
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            # EUR/USD European Special FX Option
            # - is an instrument of type FxOption,
            # - has ExerciseType feature is "European",
            # - has a property at Instrument/{scope}/special with value "yes"
            # and hence satisfies this rule and is valued with model Discounting.
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="FXOption",
                address_key_filters = [
                    lm.AddressKeyFilter("Instrument/Features/ExerciseType", "eq", lm.ResultValueString("European", result_value_type="ResultValueString")),
                    lm.AddressKeyFilter(f"Instrument/{scope}/special", "eq", lm.ResultValueString("yes", result_value_type="ResultValueString"))
                ],
                parameters="{}",
            ),
            # EUR/USD European Not Special FX Option
            # - doesn't satisfy the rule above (as it doesn't have property "Instrument/{scope}/special" with value "yes")
            # - an instrument of type FxOption,
            # - its ExerciseType feature is "European",
            # and hence satisfies this rule and is valued with model BlackScholes.
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="BlackScholes",
                instrument_type="FXOption",
                address_key_filters = [
                    lm.AddressKeyFilter("Instrument/Features/ExerciseType", "eq", lm.ResultValueString("European", result_value_type="ResultValueString"))
                ],
                parameters="{}",
            ),
            # EUR/USD American FX Option
            # - doesn't satisfy the rule above (as its feature ExerciseType is not of value "European")
            # - an instrument of type FxOption,
            # - its ExerciseType feature is "American",
            # and hence satisfies this rule and is valued with model BjerksundStensland1993.
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="BjerksundStensland1993",
                instrument_type="FXOption",
                address_key_filters = [lm.AddressKeyFilter("Instrument/Features/ExerciseType", "eq", lm.ResultValueString("American", result_value_type="ResultValueString"))],
                parameters="{}",
            ),
            # No instrument ever reaches this vendor model rule, it is never used.
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="Bachelier",
                instrument_type="FXOption",
                parameters="{}",
            ),
        ],
        options = lm.PricingOptions(
            allow_partially_successful_evaluation = True
        )
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)


print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2023-05-19 13:58:49.879004+00:00.


Note, an instrument may not support a given model, in which case a failure will be returned. To check which models are supported by a given instrument we can call `GetExistingInstrumentModels` endpoint, lets pick the "special" european option as an example:

In [25]:
supported_models = instruments_api.get_existing_instrument_models(luid_opt1, instrument_scope=scope).supported_models
print(supported_models)

['SimpleStatic', 'Discounting', 'BlackScholes', 'ConstantTimeValueOfMoney', 'Bachelier']


For more information about what pricing model can be used in your valuation please refer to https://support.lusid.com/knowledgebase/article/KA-01980/.

# 6. Valuation

Value the portfolio using the setup above

In [26]:
def run_valuation(date, portfolio_code):

    metrics = [
        lm.AggregateSpec("Instrument/default/Name", "Value"),
        lm.AggregateSpec("Valuation/PV/Amount", "Value"),
        lm.AggregateSpec("Valuation/Model/Name", "Value")
    ]

    group_by =[]

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=market_data_scope, code=recipe_code),
        metrics=metrics,
        group_by=group_by,
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregation_api.get_valuation(valuation_request=valuation_request).data
    
    vals_df = pd.DataFrame(val_data)

    return vals_df

In [27]:
valuation = run_valuation(base_date.isoformat(), portfolio_code)
display(valuation.head(3))

,Valuation/PV/Amount,Instrument/default/Name,Valuation/Model/Name
0,"5,421.1796",EUR/USD European Special FX Option 02/19/2021 ...,Discounting
1,"5,421.1796",EUR/USD European Special FX Option 02/19/2021 ...,Discounting
2,"28,082.6301",EUR/USD European Not Special FX Option 05/20/2...,BlackScholes


# 7. Features
Above we used the ExerciseType feature to help us select the desired model. Features are effectively labels attached to an instrument describing some part of it. In order to find out what features our instrument we call instrument capabilities endpoint, this endpoint contains variety of information for a given instrument. In particular we can get the features, for example picking the first instrument (Special European Fx Option) we get

In [28]:
capabilities = instruments_api.get_existing_instrument_capabilities(identifier=luid_opt1, instrument_scope=scope)
df = pd.DataFrame(list(capabilities.features.items()), columns=['AddressKey', 'Value'])

display(df)

,AddressKey,Value
0,Instrument/Features/OptionType,Call
1,Instrument/Features/DeliveryType,Physical
2,Instrument/Features/ExerciseType,European
3,Instrument/Features/PayoffType,Vanilla


In similar way we can get features for the other two FxOptions. Now we know what features our FxOption has we can setup configuration recipe as above, another question worth answering is "What possible features can an instrument of type X have? For that we can call GetAllPossibleFeatures endpoint, lets take `instrument_type=FxOption` as an example then

In [29]:
all_features = instruments_api.get_all_possible_features(instrument_type="FxOption")
df = pd.DataFrame.from_dict(all_features, orient='index')
df.fillna("", inplace=True)
df.columns = [""] * len(df.columns)

display(df)

,,,
Instrument/Features/OptionType,Call,Put,
Instrument/Features/DeliveryType,Physical,Cash,
Instrument/Features/Barrier,Double,Single,
Instrument/Features/Touch,Two,One,No
Instrument/Features/ExerciseType,American,European,
Instrument/Features/PayoffType,Digital,Vanilla,
